In [1]:
import os
import time
import datetime

from meta import *
from utils import *
from model.DataGenerator import DataGenerator
from model.NonLinearFXModel import NonLinearFXModel

import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split


instrument = GUITAR
fx = DISTORTION

input_target_file_pairs = get_input_target_fname_pairs(instrument, fx)

## PUT AUDIO Loading into a function - do it in data_loader?
n_total_clips = len(input_target_file_pairs)
input_target_pairs =  [0] * n_total_clips
for idx, (f_input, f_target) in enumerate(input_target_file_pairs):
    audio_in = load_audio(os.path.join(params_path[instrument][NO_FX], f_input), idx, n_total_clips, NO_FX)
    audio_target = load_audio(os.path.join(params_path[instrument][fx], f_target), idx, n_total_clips, fx)
    
    input_target_pairs[idx] = (audio_in, audio_target)
    
train_pairs, val_pairs = train_test_split(input_target_pairs,
                                       test_size=params_train.get('val_split'),
                                       random_state=42)


# Generate input target frame pairs
train_data_gen = DataGenerator(train_pairs, floatx=np.float32, batch_size=params_train['batch'], frame_size=params_data['frame_size'], hop_size=params_data['hop_size'], unsupervised=True)
val_data_gen = DataGenerator(train_pairs, floatx=np.float32, batch_size=params_train['batch'], frame_size=params_data['frame_size'], hop_size=params_data['hop_size'], unsupervised=True)

start = time.time()

now = datetime.datetime.now()
print("Current date and time:")
print(str(now))

tr_loss, val_loss = [0] * params_train.get('n_epochs'), [0] * params_train.get('n_epochs')

model = NonLinearFXModel(params_data=params_data, params_train=params_train, dropout=True, dropout_rate=0.3).build()
opt = Adam(lr=params_train['lr'])
model.compile(optimizer=opt, loss='mae', metrics=['mae', 'mse'])
model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=params_train.get('patience'), min_delta=0.001, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
callback_list = [early_stop, reduce_lr]


hist = model.fit_generator(train_data_gen,
               steps_per_epoch=train_data_gen.n_iterations,
               epochs=params_train.get('n_epochs'),
               validation_data=val_data_gen,
               validation_steps=val_data_gen.n_iterations,
               class_weight=None,
               workers=4,
               verbose=2,
               callbacks=callback_list)

end = time.time()
print('\n=============================Job finalized==========================================================\n')
print('Time elapsed for the job: %7.2f hours' % ((end - start) / 3600.0))
print('\n====================================================================================================\n')

path = '/import/c4dm-04/cv/models/unsupervised1/'

model.save_weights(os.path.join(path, 'model.h5'))
    
# Save the model architecture
with open(os.path.join(path, 'model.json'), 'w') as f:
    f.write(model.to_json())


loading /import/c4dm-04/cv/fxdataset/guitar/NoFX/G63-57307-1111-20704.wav
loading /import/c4dm-04/cv/fxdataset/guitar/Distortion/G63-57307-4411-37552.wav
loading 2 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-53208-1111-20848.wav
loading 2 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-53208-4411-37696.wav
loading 3 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-67412-1111-21034.wav
loading 3 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-67412-4411-37882.wav
loading 4 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-64600-1111-20658.wav
loading 4 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-64600-4411-37506.wav
loading 5 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-67603-1111-20661.wav
loading 5 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-67603-4411-37509.wav
loading 6 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-56401-1111-20789.wav
loadin

loading 45 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-58308-1111-20627.wav
loading 45 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-58308-4411-37475.wav
loading 46 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-63504-1111-20649.wav
loading 46 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-63504-4411-37497.wav
loading 47 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-67508-1111-20809.wav
loading 47 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-67508-4411-37657.wav
loading 48 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-57307-1111-21016.wav
loading 48 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-57307-4411-37864.wav
loading 49 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-71607-1111-21055.wav
loading 49 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-71607-4411-37903.wav
loading 50 / 624: nofx /import/c4dm-04/cv/fxdatase

loading 89 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-67603-1111-21207.wav
loading 89 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-67603-4411-38055.wav
loading 90 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-52302-1111-21011.wav
loading 90 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-52302-4411-37859.wav
loading 91 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-60405-1111-21105.wav
loading 91 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-60405-4411-37953.wav
loading 92 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-68604-1111-21208.wav
loading 92 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-68604-4411-38056.wav
loading 93 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-74610-1111-20824.wav
loading 93 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-74610-4411-37672.wav
loading 94 / 624: nofx /import/c4dm-04/cv/fxdatase

loading 132 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-60501-4411-37728.wav
loading 133 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-69605-1111-20663.wav
loading 133 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-69605-4411-37511.wav
loading 134 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-59309-1111-20940.wav
loading 134 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-59309-4411-37788.wav
loading 135 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-75611-1111-20825.wav
loading 135 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-75611-4411-37673.wav
loading 136 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-71607-1111-20665.wav
loading 136 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-71607-4411-37513.wav
loading 137 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-49204-1111-20844.wav
loading 137 / 624: distortion /import/c4

loading 176 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-63408-1111-20874.wav
loading 176 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-63408-4411-37722.wav
loading 177 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-59500-1111-21113.wav
loading 177 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-59500-4411-37961.wav
loading 178 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-42102-1111-21141.wav
loading 178 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-42102-4411-37989.wav
loading 179 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G63-52207-1111-20691.wav
loading 179 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G63-52207-4411-37539.wav
loading 180 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-65601-1111-20815.wav
loading 180 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-65601-4411-37663.wav
loading 181 / 624: nofx /import/c4dm-04/

...

loading 480 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-41101-1111-20828.wav
loading 480 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-41101-4411-37676.wav
loading 481 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-42102-1111-20907.wav
loading 481 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-42102-4411-37755.wav
loading 482 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-60501-1111-20958.wav
loading 482 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-60501-4411-37806.wav
loading 483 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-47107-1111-20600.wav
loading 483 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-47107-4411-37448.wav
loading 484 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-69605-1111-20819.wav
loading 484 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-69605-4411-37667.wav
loading 485 / 624: nofx /import/c4dm-04/

loading 523 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G63-52112-4411-37531.wav
loading 524 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-51206-1111-21080.wav
loading 524 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-51206-4411-37928.wav
loading 525 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-62503-1111-21194.wav
loading 525 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-62503-4411-38042.wav
loading 526 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-56211-1111-21085.wav
loading 526 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-56211-4411-37933.wav
loading 527 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-59404-1111-21104.wav
loading 527 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-59404-4411-37952.wav
loading 528 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-45200-1111-21152.wav
loading 528 / 624: distortion /import/c4

loading 567 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-46106-1111-20833.wav
loading 567 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-46106-4411-37681.wav
loading 568 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-69510-1111-20889.wav
loading 568 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-69510-4411-37737.wav
loading 569 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-64505-1111-20650.wav
loading 569 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-64505-4411-37498.wav
loading 570 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-51111-1111-20604.wav
loading 570 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-51111-4411-37452.wav
loading 571 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-52302-1111-21089.wav
loading 571 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-52302-4411-37937.wav
loading 572 / 624: nofx /import/c4dm-04/

loading 610 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-56306-4411-37629.wav
loading 611 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-75611-1111-20669.wav
loading 611 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-75611-4411-37517.wav
loading 612 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-41101-1111-20906.wav
loading 612 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-41101-4411-37754.wav
loading 613 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-67603-1111-21129.wav
loading 613 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-67603-4411-37977.wav
loading 614 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-46106-1111-21145.wav
loading 614 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-46106-4411-37993.wav
loading 615 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-59309-1111-20784.wav
loading 615 / 624: distortion /import/c4

KeyboardInterrupt: 

In [2]:
from meta import *
from utils import *

from tensorflow.keras.layers import Dense, Input, Activation, Conv1D, MaxPool1D, BatchNormalization, LocallyConnected1D, Dropout, UpSampling1D, Lambda, multiply, Reshape, ZeroPadding1D, Flatten, ELU, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2 

from model.layers import DePool1D

import tensorflow as tf

class NonLinearFXModel():
    def __init__(self, params_data=None, params_train=None, dropout=True, dropout_rate=0.5):
        self.params_data = params_data
        self.params_train = params_train
        self.dropout = dropout
        self.dropout_rate = dropout_rate
        
        self.build()
        
    def get_model(self):
        return self.model
    
    def build(self):
        input_shape = (self.params_data.get('frame_size'), 1)

        x = Input(shape=input_shape, name='input_frame')
        
        self.w1 = Conv1D(filters=128, kernel_size=64,
                   activation='linear',
                   padding='same',
                   kernel_initializer='random_uniform',
                   name='conv1')
#         self.w2 = Conv1D(filters=128, kernel_size=128,
#                    activation='softplus',
#                    kernel_initializer='random_uniform',
#                    name='conv2')
        self.w2 = [0] * 128
        for i in range(128):
            self.w2[i] = Conv1D(filters=1, kernel_size=128,
                   activation='softplus',
                   kernel_initializer='random_uniform',
                   name='conv2-{0}'.format(i))
            
        self.mp = MaxPool1D(pool_size=16, name='max_pool')
        
        z = self.frontend(x)
        z = self.latent_dnn(z)
        y = self.backend(z)
        
        self.model = Model(inputs=x, outputs=y)

        return self.model
    
    def frontend(self, x):
        # L1 128x64 filters, absolute activation
        x1 = self.w1(x)
        self.X1 = x1
        x = Lambda(lambda t: K.abs(t), name="abs_activation")(x1)

        # L2 LOCALLY CONNECTED, 128 x 128 filters, softplus activation
        x = ZeroPadding1D((64,63))(x)
        X2 = [0] * 128
        for i in range(128):
            activ = Lambda(lambda x: x[:,:,i:i+1], output_shape=(params_train.get('batch_size'), 1151, 1))(x) 
            X2[i] = self.w2[i](activ) 
        X2 = Concatenate(axis=2)(X2)
        
        x = BatchNormalization()(X2)
        x = self.mp(x) # apply max pooling layer
        
        return x
    
    def backend(self, x):
        #     x = UpSampling1D(size=16)(x)
        x = DePool1D(self.mp, size=16)(x)

        # residual connection - element-wise multiply residual and X2
        x = multiply([x, self.X1])

        x = self.dnn_saaf(x)

#         v = K.variable(w1.get_weights())

#         y = tf.nn.conv1d_transpose(x, v, output_shape=(-1, params_data.get('frame_size'), 1), strides=1, name='deconv')
        
        return x
        
    def dnn_saaf(self, x):
        # dnn saaf - 4 FC LAYERS 128 64, 64 - SOFTPLUS, 128 - locally connected SAAF
        x = Dense(128,
              kernel_initializer='random_uniform',
              kernel_regularizer=l2(1e-3),
              activation='softplus',
              name='saaf-fc1')(x)
        if self.dropout:
            x = Dropout(self.dropout_rate)(x)

        x = Dense(64,
              kernel_initializer='random_uniform',
              kernel_regularizer=l2(1e-3),
              activation='softplus',
              name='saaf-fc2')(x)
        if self.dropout: 
            x = Dropout(self.dropout_rate)(x)

        x = Dense(64,
              kernel_initializer='random_uniform',
              kernel_regularizer=l2(1e-3),
              activation='softplus',
              name='saaf-fc3')(x)
        if self.dropout:
            x = Dropout(self.dropout_rate)(x)

        x = Dense(128,
              kernel_initializer='random_uniform',
              kernel_regularizer=l2(1e-3),
              name='saaf-fc4')(x)
        x = ELU()(x) # TODO: locally connected SAAF activation

        if self.dropout:
            x = Dropout(self.dropout_rate)(x)
        
        return x

    def latent_dnn(self, z):
        z = Dense(64,
              kernel_initializer='random_uniform',
              kernel_regularizer=l2(1e-3),
              activation='softplus',
              name='dnn-1')(z)
        if self.dropout:
            z = Dropout(self.dropout_rate)(z)
            
        z = Dense(64,
              kernel_initializer='random_uniform',
              kernel_regularizer=l2(1e-3),
              activation='softplus',
              name='dnn-2')(z)
        if self.dropout:
            z = Dropout(self.dropout_rate)(z)

        return z

from tensorflow.keras.optimizers import Adam

model = NonLinearFXModel(params_data=params_data, params_train=params_train, dropout=True, dropout_rate=0.3).build()
opt = Adam(lr=params_train['lr'])
model.compile(optimizer=opt, loss='mae', metrics=['mae'])
model.summary()

ValueError: Operands could not be broadcast together with shapes (1024, 64) (1024, 128)

# Create Data Generator